In [3]:
import nibabel as nib
import numpy as np
import load_confounds
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nilearn import signal

In [4]:
atlas = nib.load('/scratch/dimuthu1/PPMI_project2/PPMI_gradients/cfg/91282_Greyordinates.dscalar.nii').get_fdata()
#cortex = clean_ts[:,(atlas[0]==1)] #Selecting just the cortex

In [5]:
print(np.unique(atlas[0]))

[ 1.  8. 10. 11. 12. 13. 16. 17. 18. 26. 28. 47. 49. 50. 51. 52. 53. 54.
 58. 60.]


In [6]:
(atlas[0])[90000]

10.0

In [7]:
#(atlas[0])[0]= 5

In [8]:
(atlas[0])[0]

1.0

In [9]:
#Cifti2Image

img = nib.load('/scratch/dimuthu1/PPMI_project2/PPMI_gradients/cfg/91282_Greyordinates.dscalar.nii')
new_img = nib.Cifti2Image(atlas, img.header)

nib.save(new_img, 'new_image.nii')

In [10]:
new_atlas = nib.load('new_image.nii').get_fdata()
print(new_atlas[0][0])


1.0


In [11]:
    true_vals = atlas[0]
    put_L = true_vals[(true_vals==12)]
    put_R = true_vals[(true_vals==51)]
    cau_L = true_vals[(true_vals==11)]
    cau_R = true_vals[(true_vals==50)]
    acc_L = true_vals[(true_vals==26)]
    acc_R = true_vals[(true_vals==58)]

    
    subctx_vals = np.concatenate((put_L, cau_L, acc_L, put_R, cau_R, acc_R),axis=0)
    print(np.size(subctx_vals))

3828


In [12]:
a = np.asarray([1,5,2,4,2,2,7,9])
b=[2,2,7,5]
res = np.where(a==2)
print(res)

(array([2, 4, 5]),)


In [13]:
true_vals = atlas[0]
put_L = np.where(true_vals==12)
put_R = np.where(true_vals==51)
cau_L = np.where(true_vals==11)
cau_R = np.where(true_vals==50)
acc_L = np.where(true_vals==26)
acc_R = np.where(true_vals==58)


subctx_vals = np.concatenate((put_L, cau_L, acc_L, put_R, cau_R, acc_R),axis=1)
print(subctx_vals[0,:])

[86676 86677 86678 ... 59684 59685 59686]


In [14]:
#for i in subctx_vals:
#    true_vals[i]=100

#print(np.unique(true_vals))

In [15]:
grads = pd.read_csv('/scratch/dimuthu1/PPMI_project2/derivatives/analysis/plots/group/sbctx_gradients.csv')

grad_12 = grads.loc[grads['month'] == 'm12']
#print(grad_12)
grad_array = grad_12['sbctx_grad_1'].values

In [16]:
np.shape(grad_array)

(3828,)

In [17]:
sbctx_ind = subctx_vals[0,:]
for i, val  in enumerate(sbctx_ind):
    true_vals[val]=grad_array[i]*100+100
    #print(true_vals[val])

print(np.unique(true_vals))

[  1.           8.          10.         ... 517.24295871 517.46880036
 524.75609769]


In [18]:
#print(true_vals)
for i, val in enumerate(true_vals):
    #print(i)
    if i not in sbctx_ind.tolist():
        true_vals[i]=-10

#print(np.unique(true_vals))

In [19]:
#print(np.shape(true_vals))
true_vals = true_vals.reshape(1,-1)
#print(np.shape(true_vals))

new_img = nib.Cifti2Image(true_vals, img.header)

nib.save(new_img, 'new_image.dscalar.nii')

In [27]:
atlas_L = nib.load('/scratch/dimuthu1/PPMI_project2/PPMI_gradients/cfg/L.atlasroi.32k_fs_LR.shape.gii')
atlas_L = atlas_L.darrays[0].data
atlas_R = nib.load('/scratch/dimuthu1/PPMI_project2/PPMI_gradients/cfg/R.atlasroi.32k_fs_LR.shape.gii')
atlas_R = atlas_R.darrays[0].data
atlas = np.hstack((atlas_L,atlas_R))

schaefer_left = nib.load('/scratch/dimuthu1/PPMI_project2/PPMI_gradients/cfg/schaefer-1000.L.32k_fs_LR.label.gii').darrays[0].data
schaefer_right = nib.load('/scratch/dimuthu1/PPMI_project2/PPMI_gradients/cfg/schaefer-1000.R.32k_fs_LR.label.gii').darrays[0].data
schaefer_LR = np.hstack((schaefer_left,schaefer_right))
labels_LR_no_medialwall = schaefer_LR[atlas==1]
print(labels_LR_no_medialwall)

[479 170 124 ... 938 938 862]


In [39]:

atlas_M = nib.load('/scratch/dimuthu1/PPMI_project2/Human.MedialWall_Conte69.32k_fs_LR.dlabel.nii')
med_val = atlas_M.get_data()
labels_with_med =np.zeros((med_val.shape))
labels_with_med[med_val != 1] = labels_LR_no_medialwall

print(labels_with_med)


pixdim[1,2,3] should be non-zero; setting 0 dims to 1
[[479. 170. 124. ... 938. 938. 862.]]
